# 1. Install modules

In [1]:
# Bioinformatics Tools (Ubuntu)
!sudo apt-get update
!sudo apt-get install -y fastp flash bwa samtools

# Python Library
!pip3 install biopython cutadapt pysam --break-system-packages

Hit:1 http://ports.ubuntu.com/ubuntu-ports noble InRelease                     
Hit:2 http://ports.ubuntu.com/ubuntu-ports noble-updates InRelease             
Hit:3 http://ports.ubuntu.com/ubuntu-ports noble-backports InRelease           
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu noble InRelease
Hit:5 http://ports.ubuntu.com/ubuntu-ports noble-security InRelease 
Hit:6 https://packages.microsoft.com/repos/code stable InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fastp is already the newest version (0.23.4+dfsg-1).
flash is already the newest version (1.2.11-2).
bwa is already the newest version (0.7.17-7).
samtools is already the newest version (1.19.2-1build2).
The following packages were automatically installed and are no longer required:
  pigz python3-xopen
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 275 not upgraded.


# 2. Trimming and Discard

In [66]:
import subprocess
import glob
import os

# Specify the folder containing your input files
# Specify the folder where you want to save the untrimmed sequences (adapter-free sequences)
input_folder = "fastq_7_8_9_10_11_12"
untrimmed_output_folder = "fastq_7_8_9_10_11_12/A_Untrimmed_output"

# Define the adapter sequences for R1 and R2
adapter_sequence_r1 = "AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC"
adapter_sequence_r2 = "AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT"

# Use glob to get a list of all input file pairs (R1 and R2) in the folder
input_file_pairs = []
for input_r1 in glob.glob(os.path.join(input_folder, "*_R1.fastq.gz")):
    # Assuming R2 files have the same naming format as R1 files
    input_r2 = input_r1.replace("_R1.fastq.gz", "_R2.fastq.gz")
    if os.path.exists(input_r2):  # Ensure R2 file exists
        input_file_pairs.append({"r1": input_r1, "r2": input_r2})

# Create the output folder if it doesn't exist
os.makedirs(untrimmed_output_folder, exist_ok=True)

for input_files in input_file_pairs:
    input_r1 = input_files["r1"]
    input_r2 = input_files["r2"]

    # Define output file paths for untrimmed (clean, adapter-free) sequences
    untrimmed_r1 = os.path.join(untrimmed_output_folder, os.path.basename(input_r1).replace(".fastq.gz", "_untrimmed.fastq.gz"))
    untrimmed_r2 = os.path.join(untrimmed_output_folder, os.path.basename(input_r2).replace(".fastq.gz", "_untrimmed.fastq.gz"))

    # Use cutadapt to keep only untrimmed sequences (completely adapter-free)
    result = subprocess.run([
        "cutadapt",
        "-a", adapter_sequence_r1,  # Adapter for R1
        "-A", adapter_sequence_r2,  # Adapter for R2
        "-O", "15",  # Minimum overlap for adapter trimming
        "--discard-trimmed",  # Discard sequences where trimming occurred
        "-o", untrimmed_r1,  # Save only untrimmed R1 reads
        "-p", untrimmed_r2,  # Save only untrimmed R2 reads
        input_r1, input_r2
    ], capture_output=True, text=True)

    # Log result
    if result.returncode == 0:
        print(f"Untrimmed sequences saved: {untrimmed_r1}, {untrimmed_r2}")
    else:
        print(f"Error processing {input_r1} and {input_r2}:\n{result.stderr}")

Untrimmed sequences saved: fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_07step_R1_untrimmed.fastq.gz, fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_07step_R2_untrimmed.fastq.gz
Untrimmed sequences saved: fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_08step_R1_untrimmed.fastq.gz, fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_08step_R2_untrimmed.fastq.gz
Untrimmed sequences saved: fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_09step_R1_untrimmed.fastq.gz, fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_09step_R2_untrimmed.fastq.gz
Untrimmed sequences saved: fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_10step_R1_untrimmed.fastq.gz, fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_10step_R2_untrimmed.fastq.gz


# 3. Q filtering

In [67]:
import os
import subprocess

# Quality threshold (Phred score)
quality_threshold = 30

# Set input and output folders
input_folder = "fastq_7_8_9_10_11_12/A_Untrimmed_output"
output_folder = "fastq_7_8_9_10_11_12/B_Qfiltered"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)  

# Iterate over files in the input folder; process only those ending with "_untrimmed.fastq.gz"
for filename in os.listdir(input_folder):
    if filename.endswith("_untrimmed.fastq.gz"):
        # Input file path
        input_file = os.path.join(input_folder, filename)
        
        # Output file name (e.g., sample_untrimmed.fastq.gz -> sample_Qfiltered.fastq.gz)
        output_file = os.path.join(
            output_folder, 
            filename.replace("_untrimmed.fastq.gz", "_Qfiltered.fastq.gz")
        )
        
        # Run fastp (single-end mode)
        subprocess.call([
            "fastp",
            "-i", input_file,                 # Input file
            "-o", output_file,                # Output file
            "-q", str(quality_threshold),     # Remove bases below Q30
            "-u", "15",                       # Discard reads if >15% of bases are low quality
            "-l", "151",                      # Minimum read length
            "--cut_mean_quality", "30",       # Discard reads with mean quality < 30
            "--html", f"{output_file}.html",  # HTML report
            "--json", f"{output_file}.json"   # JSON report
        ])
        
        print(f"Filtering for {filename} is complete.\n"
              f"Output FASTQ : {output_file}\n"
              f"Reports      : {output_file}.html / {output_file}.json\n")

print("All filtering processes are done.")

Detecting adapter sequence for read1...
No adapter detected for read1

Read1 before filtering:
total reads: 1695
total bases: 255945
Q20 bases: 216727(84.6772%)
Q30 bases: 192828(75.3396%)

Read1 after filtering:
total reads: 572
total bases: 86372
Q20 bases: 83774(96.9921%)
Q30 bases: 79404(91.9326%)

Filtering result:
reads passed filter: 572
reads failed due to low quality: 1123
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 18.6431%

JSON report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R1_Qfiltered.fastq.gz.json
HTML report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R1_Qfiltered.fastq.gz.html

fastp -i fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_10step_R1_untrimmed.fastq.gz -o fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R1_Qfiltered.fastq.gz -q 30 -u 15 -l 151 --cut_mean_quality 30 --html fastq_7_8_9_10_11_12/B_

Filtering for Stepwise_10step_R1_untrimmed.fastq.gz is complete.
Output FASTQ : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R1_Qfiltered.fastq.gz
Reports      : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R1_Qfiltered.fastq.gz.html / fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R1_Qfiltered.fastq.gz.json



Read1 before filtering:
total reads: 1695
total bases: 255945
Q20 bases: 216366(84.5361%)
Q30 bases: 190719(74.5156%)

Read1 after filtering:
total reads: 430
total bases: 64930
Q20 bases: 62401(96.105%)
Q30 bases: 58299(89.7875%)

Filtering result:
reads passed filter: 430
reads failed due to low quality: 1265
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 12.6844%

JSON report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R2_Qfiltered.fastq.gz.json
HTML report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R2_Qfiltered.fastq.gz.html

fastp -i fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_10step_R2_untrimmed.fastq.gz -o fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R2_Qfiltered.fastq.gz -q 30 -u 15 -l 151 --cut_mean_quality 30 --html fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R2_Qfiltered.fastq.gz.html --json fastq_7_8_9_

Filtering for Stepwise_10step_R2_untrimmed.fastq.gz is complete.
Output FASTQ : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R2_Qfiltered.fastq.gz
Reports      : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R2_Qfiltered.fastq.gz.html / fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_10step_R2_Qfiltered.fastq.gz.json



Read1 before filtering:
total reads: 1829
total bases: 276179
Q20 bases: 240704(87.1551%)
Q30 bases: 215126(77.8937%)

Read1 after filtering:
total reads: 696
total bases: 105096
Q20 bases: 101193(96.2863%)
Q30 bases: 94970(90.365%)

Filtering result:
reads passed filter: 696
reads failed due to low quality: 1133
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 21.3778%

JSON report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R2_Qfiltered.fastq.gz.json
HTML report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R2_Qfiltered.fastq.gz.html

fastp -i fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_08step_R2_untrimmed.fastq.gz -o fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R2_Qfiltered.fastq.gz -q 30 -u 15 -l 151 --cut_mean_quality 30 --html fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R2_Qfiltered.fastq.gz.html --json fastq_7_8_

...

Read1 before filtering:
total reads: 1829
total bases: 276179
Q20 bases: 242175(87.6877%)
Q30 bases: 218794(79.2218%)

Read1 after filtering:
total reads: 838
total bases: 126538
Q20 bases: 123192(97.3557%)
Q30 bases: 117361(92.7476%)

Filtering result:
reads passed filter: 838
reads failed due to low quality: 991
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 31.5473%

JSON report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R1_Qfiltered.fastq.gz.json
HTML report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R1_Qfiltered.fastq.gz.html

fastp -i fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_08step_R1_untrimmed.fastq.gz -o fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R1_Qfiltered.fastq.gz -q 30 -u 15 -l 151 --cut_mean_quality 30 --html fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R1_Qfiltered.fastq.gz.html --json fastq_7_8

Filtering for Stepwise_08step_R1_untrimmed.fastq.gz is complete.
Output FASTQ : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R1_Qfiltered.fastq.gz
Reports      : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R1_Qfiltered.fastq.gz.html / fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_08step_R1_Qfiltered.fastq.gz.json



Read1 before filtering:
total reads: 1956
total bases: 295356
Q20 bases: 256879(86.9727%)
Q30 bases: 231776(78.4734%)

Read1 after filtering:
total reads: 852
total bases: 128652
Q20 bases: 125281(97.3798%)
Q30 bases: 119457(92.8528%)

Filtering result:
reads passed filter: 852
reads failed due to low quality: 1104
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 27.2495%

JSON report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R1_Qfiltered.fastq.gz.json
HTML report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R1_Qfiltered.fastq.gz.html

fastp -i fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_09step_R1_untrimmed.fastq.gz -o fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R1_Qfiltered.fastq.gz -q 30 -u 15 -l 151 --cut_mean_quality 30 --html fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R1_Qfiltered.fastq.gz.html --json fastq_7_

Filtering for Stepwise_09step_R1_untrimmed.fastq.gz is complete.
Output FASTQ : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R1_Qfiltered.fastq.gz
Reports      : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R1_Qfiltered.fastq.gz.html / fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R1_Qfiltered.fastq.gz.json

Filtering for Stepwise_09step_R2_untrimmed.fastq.gz is complete.
Output FASTQ : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R2_Qfiltered.fastq.gz
Reports      : fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R2_Qfiltered.fastq.gz.html / fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R2_Qfiltered.fastq.gz.json

All filtering processes are done.


Read1 before filtering:
total reads: 1956
total bases: 295356
Q20 bases: 254346(86.1151%)
Q30 bases: 226989(76.8527%)

Read1 after filtering:
total reads: 692
total bases: 104492
Q20 bases: 100693(96.3643%)
Q30 bases: 94792(90.717%)

Filtering result:
reads passed filter: 692
reads failed due to low quality: 1264
reads failed due to too many N: 0
reads failed due to too short: 0
reads with adapter trimmed: 0
bases trimmed due to adapters: 0

Duplication rate (may be overestimated since this is SE data): 17.0245%

JSON report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R2_Qfiltered.fastq.gz.json
HTML report: fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R2_Qfiltered.fastq.gz.html

fastp -i fastq_7_8_9_10_11_12/A_Untrimmed_output/Stepwise_09step_R2_untrimmed.fastq.gz -o fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R2_Qfiltered.fastq.gz -q 30 -u 15 -l 151 --cut_mean_quality 30 --html fastq_7_8_9_10_11_12/B_Qfiltered/Stepwise_09step_R2_Qfiltered.fastq.gz.html --json fastq_7_8_

# 4. Match Paired-End Read IDs

In [68]:
import gzip
import glob
import os

def extract_matching_reads(r1_path, r2_path, out_r1_path, out_r2_path):
    def get_read_id(header):
        # Extract ID from FASTQ header
        return header.split()[0].replace('/1', '').replace('/2', '')

    r1_ids = set()
    r2_ids = set()

    with gzip.open(r1_path, 'rt') as r1_file:
        while True:
            header = r1_file.readline()
            if not header:
                break
            r1_ids.add(get_read_id(header.strip()))
            # Skip the other 3 lines of the read (sequence, +, quality)
            [r1_file.readline() for _ in range(3)]  

    with gzip.open(r2_path, 'rt') as r2_file:
        while True:
            header = r2_file.readline()
            if not header:
                break
            r2_ids.add(get_read_id(header.strip()))
            [r2_file.readline() for _ in range(3)]

    matching_ids = r1_ids & r2_ids
    r1_only = r1_ids - r2_ids
    r2_only = r2_ids - r1_ids

    print(f"Processing {os.path.basename(r1_path)} and {os.path.basename(r2_path)}")
    print(f"Total R1 IDs: {len(r1_ids)}, Total R2 IDs: {len(r2_ids)}, Matching IDs: {len(matching_ids)}")
    print(f"IDs only in R1: {len(r1_only)}, IDs only in R2: {len(r2_only)}\n")

    # Create output directories if needed
    for out_path in [out_r1_path, out_r2_path]:
        os.makedirs(os.path.dirname(out_path), exist_ok=True)

    # Function to write only the reads with matching IDs to a new file
    def write_matching_reads(input_path, output_path, matching_ids):
        with gzip.open(input_path, 'rt') as infile, gzip.open(output_path, 'wt') as outfile:
            while True:
                lines = [infile.readline() for _ in range(4)]
                if not lines[0]:
                    break
                read_id = get_read_id(lines[0].strip())
                if read_id in matching_ids:
                    outfile.writelines(lines)
 
    # Write the filtered R1 and R2 files
    write_matching_reads(r1_path, out_r1_path, matching_ids)
    write_matching_reads(r2_path, out_r2_path, matching_ids)

# ----------------------
# Apply to all files
# ----------------------

input_folder = "fastq_7_8_9_10_11_12/B_Qfiltered"
output_folder = "fastq_7_8_9_10_11_12/C_ID_matched"

# Find all R1 files
r1_files = glob.glob(os.path.join(input_folder, "*_R1_Qfiltered.fastq.gz"))

# For each R1, find the matching R2 and process
for r1_file in r1_files:
    r2_file = r1_file.replace("_R1_Qfiltered.fastq.gz", "_R2_Qfiltered.fastq.gz")
    
    if os.path.exists(r2_file):
        # Set output paths
        base_name = os.path.basename(r1_file).replace("_R1_Qfiltered.fastq.gz", "")
        out_r1 = os.path.join(output_folder, f"{base_name}_ID_match_R1.fastq.gz")
        out_r2 = os.path.join(output_folder, f"{base_name}_ID_match_R2.fastq.gz")
        
        # Execute the function
        extract_matching_reads(r1_file, r2_file, out_r1, out_r2)
    else:
        print(f"Warning: {r2_file} not found. Skipping.")

Processing Stepwise_09step_R1_Qfiltered.fastq.gz and Stepwise_09step_R2_Qfiltered.fastq.gz
Total R1 IDs: 852, Total R2 IDs: 692, Matching IDs: 605
IDs only in R1: 247, IDs only in R2: 87

Processing Stepwise_10step_R1_Qfiltered.fastq.gz and Stepwise_10step_R2_Qfiltered.fastq.gz
Total R1 IDs: 572, Total R2 IDs: 430, Matching IDs: 349
IDs only in R1: 223, IDs only in R2: 81

Processing Stepwise_08step_R1_Qfiltered.fastq.gz and Stepwise_08step_R2_Qfiltered.fastq.gz
Total R1 IDs: 838, Total R2 IDs: 696, Matching IDs: 590
IDs only in R1: 248, IDs only in R2: 106

Processing Stepwise_07step_R1_Qfiltered.fastq.gz and Stepwise_07step_R2_Qfiltered.fastq.gz
Total R1 IDs: 1042, Total R2 IDs: 909, Matching IDs: 814
IDs only in R1: 228, IDs only in R2: 95



# 5. Merge W/ Flash

## 5.1. DNA Fragmentation R1(Front, Back), R2(Front, Back)

In [69]:
import os
import gzip
import glob

def split_fastq_by_position(r1_path, r2_path, n, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    sample_base = os.path.basename(r1_path).replace("_ID_match_R1.fastq.gz", "")
    r1_f_path = os.path.join(output_dir, f"{sample_base}_R1_F.fastq.gz")
    r1_b_path = os.path.join(output_dir, f"{sample_base}_R1_B.fastq.gz")
    r2_f_path = os.path.join(output_dir, f"{sample_base}_R2_F.fastq.gz")
    r2_b_path = os.path.join(output_dir, f"{sample_base}_R2_B.fastq.gz")

    with gzip.open(r1_path, 'rt') as r1_file, \
         gzip.open(r2_path, 'rt') as r2_file, \
         gzip.open(r1_f_path, 'wt') as r1_f_out, \
         gzip.open(r1_b_path, 'wt') as r1_b_out, \
         gzip.open(r2_f_path, 'wt') as r2_f_out, \
         gzip.open(r2_b_path, 'wt') as r2_b_out:

        while True:
            r1_lines = [r1_file.readline() for _ in range(4)]
            r2_lines = [r2_file.readline() for _ in range(4)]

            if not r1_lines[0] or not r2_lines[0]:
                break

            header1, seq1, plus1, qual1 = [line.strip() for line in r1_lines]
            header2, seq2, plus2, qual2 = [line.strip() for line in r2_lines]

            r1_f_out.write(f"{header1}\n{seq1[:151-n]}\n{plus1}\n{qual1[:151-n]}\n")
            r1_b_out.write(f"{header1}\n{seq1[-n:]}\n{plus1}\n{qual1[-n:]}\n")
            r2_f_out.write(f"{header2}\n{seq2[:151-n]}\n{plus2}\n{qual2[:151-n]}\n")
            r2_b_out.write(f"{header2}\n{seq2[-n:]}\n{plus2}\n{qual2[-n:]}\n")

    print(f"✅ Split complete: {sample_base} → {output_dir} (N={n})")

# -----------------------------------
# Apply splitting to all files
# -----------------------------------

input_folder = "fastq_7_8_9_10_11_12/C_id_matched"
output_folder = "fastq_7_8_9_10_11_12/D_split_reads"
os.makedirs(output_folder, exist_ok=True)

# Define the N-value (length of the back part) for each sample prefix
sample_n_mapping = {
    "07step": 144,
    "08step": 124,
    "09step": 102,
    "10step": 82,
    "11step": 60,
    "12step": 40, 
}

# Find all R1 files
r1_files = glob.glob(os.path.join(input_folder, "*_ID_match_R1.fastq.gz"))

for r1_file in r1_files:
    r2_file = r1_file.replace("_R1.fastq.gz", "_R2.fastq.gz")

    if not os.path.exists(r2_file):
        print(f"⚠️ Matching R2 file not found: {r2_file}")
        continue

    # Find the corresponding N value based on the filename prefix
    matched_n = None
    for prefix, n_value in sample_n_mapping.items():
        if prefix in os.path.basename(r1_file):
            matched_n = n_value
            break

    if matched_n is None:
        print(f"⚠️ Could not find N value: {r1_file} → Skipping")
        continue

    # Execute the split function
    split_fastq_by_position(r1_file, r2_file, matched_n, output_folder)

✅ Split complete: Stepwise_10step → fastq_7_8_9_10_11_12/D_split_reads (N=82)
✅ Split complete: Stepwise_07step → fastq_7_8_9_10_11_12/D_split_reads (N=144)
✅ Split complete: Stepwise_09step → fastq_7_8_9_10_11_12/D_split_reads (N=102)
✅ Split complete: Stepwise_08step → fastq_7_8_9_10_11_12/D_split_reads (N=124)


# 5.2. R2 DNA reverse complementary

In [70]:
import gzip
import glob
import os
from Bio import SeqIO

def reverse_complement_fastq(input_fastq_path, output_fastq_path):
    #Reads a FASTQ file, creates the reverse complement of each record, and writes it to a new file.
    with gzip.open(input_fastq_path, "rt") as infile, gzip.open(output_fastq_path, "wt") as outfile:
        for record in SeqIO.parse(infile, "fastq"):
            record.seq = record.seq.reverse_complement()
            record.letter_annotations["phred_quality"] = record.letter_annotations["phred_quality"][::-1]
            SeqIO.write(record, outfile, "fastq")
    print(f"✅ Reverse complemented: {os.path.basename(output_fastq_path)}")

# --------------------------------------
# Apply reverse complement to all files
# --------------------------------------

input_folder = "fastq_7_8_9_10_11_12/D_split_reads"
os.makedirs(input_folder, exist_ok=True)

# Find files ending with _R2_B.fastq.gz or _R2_F.fastq.gz
input_files = glob.glob(os.path.join(input_folder, "*_R2_[BF].fastq.gz"))

for input_path in input_files:
    base = os.path.basename(input_path)
    # Remove the .fastq.gz extension to create a new filename
    name_without_ext = base.replace(".fastq.gz", "") 
    output_path = os.path.join(input_folder, f"{name_without_ext}_revcomp.fastq.gz")
    
    reverse_complement_fastq(input_path, output_path)

✅ Reverse complemented: Stepwise_07step_R2_F_revcomp.fastq.gz
✅ Reverse complemented: Stepwise_08step_R2_B_revcomp.fastq.gz
✅ Reverse complemented: Stepwise_10step_R2_B_revcomp.fastq.gz
✅ Reverse complemented: Stepwise_09step_R2_B_revcomp.fastq.gz
✅ Reverse complemented: Stepwise_08step_R2_F_revcomp.fastq.gz
✅ Reverse complemented: Stepwise_09step_R2_F_revcomp.fastq.gz
✅ Reverse complemented: Stepwise_10step_R2_F_revcomp.fastq.gz
✅ Reverse complemented: Stepwise_07step_R2_B_revcomp.fastq.gz


## 5.3 [R1_back]-[R2_back] merge (FLASH)

In [71]:
import os
import glob
import subprocess

# === Folder setup ===
input_folder = "fastq_7_8_9_10_11_12/D_split_reads"
output_folder = "fastq_7_8_9_10_11_12/E_merged_output"
os.makedirs(output_folder, exist_ok=True)

# === N values per prefix ===
sample_n_mapping = {
    "07step": 144,
    "08step": 124,
    "09step": 102,
    "10step": 82,
    "11step": 60,
    "12step": 40, 
}

# === Find R1_B files ===
r1_files = glob.glob(os.path.join(input_folder, "*_R1_B.fastq.gz"))
print(f"🔎 Found {len(r1_files)} R1_B files.")

# === Process each R1_B file ===
for r1_path in r1_files:
    sample_base = os.path.basename(r1_path).replace("_R1_B.fastq.gz", "")
    r2_path = os.path.join(input_folder, f"{sample_base}_R2_B.fastq.gz")

    if not os.path.exists(r2_path):
        print(f"⚠️ Matching R2_B file not found for {sample_base} → Skipping.")
        continue

    # Find the corresponding N value from the filename
    matched_n = None
    for prefix, n_value in sample_n_mapping.items():
        if prefix in sample_base:
            matched_n = n_value
            break

    if matched_n is None:
        print(f"⚠️ No N value matched for {sample_base} → Skipping.")
        continue

    output_name = f"{sample_base}_FLASH"

    print(f"🔵 Running FLASH for sample: {sample_base} (N={matched_n})")

    try:
        # Execute the FLASH command
        subprocess.check_call([
            "flash",
            "-m", str(matched_n),   # minimum overlap
            "-M", str(matched_n),   # maximum overlap
            "-o", output_name,      # output file prefix
            "-d", output_folder,    # output directory
            r1_path,
            r2_path
        ])
        print(f"✅ FLASH merging complete → {os.path.join(output_folder, output_name)}.fastq")
    except subprocess.CalledProcessError as e:
        print(f"❌ FLASH merging failed for {sample_base}: {e}")

🔎 Found 4 R1_B files.
🔵 Running FLASH for sample: Stepwise_07step (N=144)
[FLASH] Starting FLASH v1.2.11
[FLASH] Fast Length Adjustment of SHort reads
[FLASH]  
[FLASH] Input files:
[FLASH]     fastq_7_8_9_10_11_12/D_split_reads/Stepwise_07step_R1_B.fastq.gz
[FLASH]     fastq_7_8_9_10_11_12/D_split_reads/Stepwise_07step_R2_B.fastq.gz
[FLASH]  
[FLASH] Output files:
[FLASH]     fastq_7_8_9_10_11_12/E_merged_output/Stepwise_07step_FLASH.extendedFrags.fastq
[FLASH]     fastq_7_8_9_10_11_12/E_merged_output/Stepwise_07step_FLASH.notCombined_1.fastq
[FLASH]     fastq_7_8_9_10_11_12/E_merged_output/Stepwise_07step_FLASH.notCombined_2.fastq
[FLASH]     fastq_7_8_9_10_11_12/E_merged_output/Stepwise_07step_FLASH.hist
[FLASH]     fastq_7_8_9_10_11_12/E_merged_output/Stepwise_07step_FLASH.histogram
[FLASH]  
[FLASH] Parameters:
[FLASH]     Min overlap:           144
[FLASH]     Max overlap:           144
[FLASH]     Max mismatch density:  0.250000
[FLASH]     Allow "outie" pairs:   false
[FLASH]  

## 5.4 Assemble 
## R1_Front - [R1_Back]-[R2_Back]_merged (FLASH) - R2_Front_ReverseComplement

In [72]:
import os
import gzip
import glob
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def load_fastq_to_dict(file_path):
    """Loads a FASTQ file into a dictionary: key=read_id, value=(sequence, quality)."""
    data = {}
    open_func = gzip.open if file_path.endswith(".gz") else open

    with open_func(file_path, "rt") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            seq = str(record.seq)
            qual = record.letter_annotations["phred_quality"]
            data[record.id] = (seq, qual)
    return data

def assemble_fastq(r1_path, merged_path, r2_path, output_path):
    """Assembles the final sequence from R1_F, Merged, and R2_F_revcomp fragments."""
    print(f"🔄 Assembling for sample: {os.path.basename(output_path)}")
    r1_dict = load_fastq_to_dict(r1_path)
    r2_dict = load_fastq_to_dict(r2_path)

    with open(merged_path, "r") as merged_file, gzip.open(output_path, "wt") as output_file:
        for record in SeqIO.parse(merged_file, "fastq"):
            read_id = record.id
            merged_seq = str(record.seq)
            merged_qual = record.letter_annotations["phred_quality"]

            # A read must have corresponding R1 and R2 fragments to be assembled.
            if read_id not in r1_dict or read_id not in r2_dict:
                continue  

            r1_seq, r1_qual = r1_dict[read_id]
            r2_seq, r2_qual = r2_dict[read_id]

            # Concatenate in order: R1_F → Merged_Fragment → R2_F_revcomp
            full_seq = r1_seq + merged_seq + r2_seq
            full_qual = r1_qual + merged_qual + r2_qual

            new_record = SeqRecord(
                Seq(full_seq),
                id=read_id,
                description="",
                letter_annotations={"phred_quality": full_qual}
            )

            SeqIO.write(new_record, output_file, "fastq")

    print(f"✅ Assembled FASTQ saved: {output_path}")

# ===== Batch processing =====

# Path setup
input_merged_folder = "fastq_7_8_9_10_11_12/E_merged_output"
input_split_folder = "fastq_7_8_9_10_11_12/D_split_reads"
output_folder = "fastq_7_8_9_10_11_12/1_assemble"
os.makedirs(output_folder, exist_ok=True)

# Get a list of all merged files from FLASH
merged_files = glob.glob(os.path.join(input_merged_folder, "*_FLASH.extendedFrags.fastq"))

print(f"🔍 Found {len(merged_files)} merged samples to assemble.")

for merged_file in merged_files:
    sample_base = os.path.basename(merged_file).replace("_FLASH.extendedFrags.fastq", "")

    r1_path = os.path.join(input_split_folder, f"{sample_base}_R1_F.fastq.gz")
    r2_path = os.path.join(input_split_folder, f"{sample_base}_R2_F_revcomp.fastq.gz")
    output_path = os.path.join(output_folder, f"{sample_base}_assemble.fastq.gz")

    if os.path.exists(r1_path) and os.path.exists(r2_path):
        assemble_fastq(r1_path, merged_file, r2_path, output_path)
    else:
        print(f"⚠️ Missing split files for {sample_base}, skipping.")

🔍 Found 4 merged samples to assemble.
🔄 Assembling for sample: Stepwise_09step_assemble.fastq.gz
✅ Assembled FASTQ saved: fastq_7_8_9_10_11_12/1_assemble/Stepwise_09step_assemble.fastq.gz
🔄 Assembling for sample: Stepwise_10step_assemble.fastq.gz
✅ Assembled FASTQ saved: fastq_7_8_9_10_11_12/1_assemble/Stepwise_10step_assemble.fastq.gz
🔄 Assembling for sample: Stepwise_08step_assemble.fastq.gz
✅ Assembled FASTQ saved: fastq_7_8_9_10_11_12/1_assemble/Stepwise_08step_assemble.fastq.gz
🔄 Assembling for sample: Stepwise_07step_assemble.fastq.gz
✅ Assembled FASTQ saved: fastq_7_8_9_10_11_12/1_assemble/Stepwise_07step_assemble.fastq.gz


# 6. fastq -> fasta

In [73]:
import os
import gzip
from Bio import SeqIO

# Input and output folder paths
input_folder = "fastq_7_8_9_10_11_12/1_assemble"
output_folder = "fastq_7_8_9_10_11_12/2_fastq_to_fasta"
# Create the output folder if it doesn't exist.
os.makedirs(output_folder, exist_ok=True) 

for filename in os.listdir(input_folder):
    # Process only files with .fastq or .fastq.gz extensions
    if filename.endswith(".fastq") or filename.endswith(".fastq.gz"):
        input_file = os.path.join(input_folder, filename)
        
        # Set output filename (.fasta extension)
        output_file = os.path.join(
            output_folder,
            filename.replace(".fastq.gz", ".fasta").replace(".fastq", ".fasta")
        )

        # Choose open mode based on gzip
        open_func = gzip.open if filename.endswith(".gz") else open

        # Read FASTQ and convert to FASTA
        with open_func(input_file, "rt") as fastq_file:  # open in text mode
            records = list(SeqIO.parse(fastq_file, "fastq"))

        # Save as FASTA
        with open(output_file, "w") as fasta_file:
            SeqIO.write(records, fasta_file, "fasta")

        print(f"Converted: {filename} → {os.path.basename(output_file)}")

print("All conversions are done.")

Converted: Stepwise_07step_assemble.fastq.gz → Stepwise_07step_assemble.fasta
Converted: Stepwise_09step_assemble.fastq.gz → Stepwise_09step_assemble.fasta
Converted: Stepwise_10step_assemble.fastq.gz → Stepwise_10step_assemble.fasta
Converted: Stepwise_08step_assemble.fastq.gz → Stepwise_08step_assemble.fasta
All conversions are done.


# 7. Binary data reference seqeunce data generate

Binary data reference seqeunce was already generated

# 8. Reference sequence - Sample Matching

In [74]:
%%bash
set -euo pipefail
shopt -s nullglob

# ========== Configuration ==========
ref_dir="step_reference"                       # Reference FASTA directory
query_dir="fastq_7_8_9_10_11_12/2_fastq_to_fasta"  # Input FASTA directory
output_dir="fastq_7_8_9_10_11_12/3_align_sam"      # Output SAM directory
threads=4

# Optional: filter steps (e.g., "07"–"12")
# Leave empty to process all steps automatically.
step_min="07"   # e.g., "07"
step_max="12"   # e.g., "12"
# ====================================

mkdir -p "$output_dir"

# Avoid duplicate indexing during the run
declare -A indexed
# Avoid duplicate processing: prefer "*assemble.fasta"
declare -A seen

# 1) Collect "*assemble.fasta" first
for f in "$query_dir"/*step*assemble.fasta; do
  seen["$f"]=1
done

# 2) Collect other *step*.fasta only if not already included
for f in "$query_dir"/*step*.fasta; do
  [[ -n "${seen[$f]:-}" ]] && continue
  seen["$f"]=1
done

# Alignment loop
for query_file in "${!seen[@]}"; do
  filename="$(basename "$query_file")"

  # Extract step digit(s): matches both _07step_ and _7step
  if [[ "$filename" =~ _([0-9]+)step(_|$) ]]; then
    step_raw="${BASH_REMATCH[1]}"
  else
    echo "⚠️  Step number not found in filename: $filename"
    continue
  fi

  # Zero-padding to 2 digits (e.g., 7 → 07)
  step_pad=$(printf "%02d" $((10#$step_raw)))

  # Optional step range filtering
  if [[ -n "$step_min" && -n "$step_max" ]]; then
    s_val=$((10#$step_pad))
    s_min=$((10#$step_min))
    s_max=$((10#$step_max))
    if (( s_val < s_min || s_val > s_max )); then
      echo "⏭️  Skip (step not in ${step_min}–${step_max}): $filename"
      continue
    fi
  fi

  reference_file="${ref_dir}/${step_pad}step_reference.fasta"
  if [[ ! -f "$reference_file" ]]; then
    echo "⚠️  Missing reference: $reference_file"
    continue
  fi

  out_file="${output_dir}/${filename%.fasta}.sam"
  echo "🔄 Aligning: $filename → $(basename "$reference_file") (step=$step_pad)"

  # Index only once per reference in this run, and reuse existing index files
  if [[ -z "${indexed[$reference_file]:-}" ]]; then
    if [[ -f "${reference_file}.bwt" ]]; then
      echo "⏭️  Index exists, skipping indexing."
    else
      echo "🔧 Indexing reference..."
      bwa index "$reference_file"
    fi
    indexed[$reference_file]=1
  fi

  bwa mem -M -t "$threads" "$reference_file" "$query_file" > "$out_file"
  echo "✅ Done: $out_file"
done

🔄 Aligning: Stepwise_10step_assemble.fasta → 10step_reference.fasta (step=10)
🔧 Indexing reference...


[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.01 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.01 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index step_reference/10step_reference.fasta
[main] Real time: 0.102 sec; CPU: 0.029 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 44 sequences (9680 bp)...
[M::mem_process_seqs] Processed 44 reads in 0.109 CPU sec, 0.037 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -M -t 4 step_reference/10step_reference.fasta fastq_7_8_9_10_11_12/2_fastq_to_fasta/Stepwise_10step_assemble.fasta
[main] Real time: 0.075 sec; CPU: 0.114 sec


✅ Done: fastq_7_8_9_10_11_12/3_align_sam/Stepwise_10step_assemble.sam
🔄 Aligning: Stepwise_09step_assemble.fasta → 09step_reference.fasta (step=09)
🔧 Indexing reference...


[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.01 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index step_reference/09step_reference.fasta
[main] Real time: 0.070 sec; CPU: 0.015 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 161 sequences (32200 bp)...
[M::mem_process_seqs] Processed 161 reads in 0.238 CPU sec, 0.081 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -M -t 4 step_reference/09step_reference.fasta fastq_7_8_9_10_11_12/2_fastq_to_fasta/Stepwise_09step_assemble.fasta
[main] Real time: 0.113 sec; CPU: 0.243 sec


✅ Done: fastq_7_8_9_10_11_12/3_align_sam/Stepwise_09step_assemble.sam
🔄 Aligning: Stepwise_07step_assemble.fasta → 07step_reference.fasta (step=07)
🔧 Indexing reference...


[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index step_reference/07step_reference.fasta
[main] Real time: 0.040 sec; CPU: 0.006 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 437 sequences (69046 bp)...
[M::mem_process_seqs] Processed 437 reads in 0.103 CPU sec, 0.034 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -M -t 4 step_reference/07step_reference.fasta fastq_7_8_9_10_11_12/2_fastq_to_fasta/Stepwise_07step_assemble.fasta
[main] Real time: 0.070 sec; CPU: 0.107 sec


✅ Done: fastq_7_8_9_10_11_12/3_align_sam/Stepwise_07step_assemble.sam
🔄 Aligning: Stepwise_08step_assemble.fasta → 08step_reference.fasta (step=08)
🔧 Indexing reference...


[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.00 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index step_reference/08step_reference.fasta
[main] Real time: 0.056 sec; CPU: 0.009 sec
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 127 sequences (22606 bp)...
[M::mem_process_seqs] Processed 127 reads in 0.035 CPU sec, 0.012 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem -M -t 4 step_reference/08step_reference.fasta fastq_7_8_9_10_11_12/2_fastq_to_fasta/Stepwise_08step_assemble.fasta
[main] Real time: 0.047 sec; CPU: 0.038 sec


✅ Done: fastq_7_8_9_10_11_12/3_align_sam/Stepwise_08step_assemble.sam


## 8.1 sam to bam

In [75]:
%%bash

# Set the path to the directory containing SAM files
sam_dir="fastq_7_8_9_10_11_12/3_align_sam"
# Set the output directory for BAM files
bam_dir="fastq_7_8_9_10_11_12/4_align_bam"


# Make sure the output directory exists or create it if necessary
mkdir -p "$bam_dir"

# Convert SAM files to BAM
for sam_file in "$sam_dir"/*.sam; do
    bam_file="$bam_dir/$(basename "$sam_file" .sam).bam"
    samtools view -bS "$sam_file" -o "$bam_file"
    echo "Conversion from $sam_file to $bam_file is complete."
done

Conversion from fastq_7_8_9_10_11_12/3_align_sam/Stepwise_07step_assemble.sam to fastq_7_8_9_10_11_12/4_align_bam/Stepwise_07step_assemble.bam is complete.
Conversion from fastq_7_8_9_10_11_12/3_align_sam/Stepwise_08step_assemble.sam to fastq_7_8_9_10_11_12/4_align_bam/Stepwise_08step_assemble.bam is complete.
Conversion from fastq_7_8_9_10_11_12/3_align_sam/Stepwise_09step_assemble.sam to fastq_7_8_9_10_11_12/4_align_bam/Stepwise_09step_assemble.bam is complete.
Conversion from fastq_7_8_9_10_11_12/3_align_sam/Stepwise_10step_assemble.sam to fastq_7_8_9_10_11_12/4_align_bam/Stepwise_10step_assemble.bam is complete.


## 8.2  Convert BAM to CSV

In [76]:
import os
import pysam
import pandas as pd

# Input folder (path containing BAM files)
input_folder = "fastq_7_8_9_10_11_12/4_align_bam"
# Output folder (path to save CSV files; change if needed)
output_folder = "fastq_7_8_9_10_11_12/5_align_csv"

# Create output folder if it does not exist
os.makedirs(output_folder, exist_ok=True)

# BAM -> CSV conversion function (including optional SAM tags)
def bam_to_csv(bam_file, output_folder):
    output_csv = os.path.join(output_folder, os.path.basename(bam_file).replace(".bam", ".csv"))
    
    # Read BAM file
    with pysam.AlignmentFile(bam_file, "rb") as bam:
        records = []
        all_tags = set()  # set to collect optional tag names
        
        for read in bam:
            # Core fields
            record = {
                "QNAME": read.query_name,
                "FLAG": read.flag,
                "RNAME": bam.get_reference_name(read.reference_id) if read.reference_id >= 0 else "*",
                "POS": read.reference_start + 1,
                "MAPQ": read.mapping_quality,
                "CIGAR": read.cigarstring if read.cigarstring else "*",
                "RNEXT": bam.get_reference_name(read.next_reference_id) if read.next_reference_id >= 0 else "*",
                "PNEXT": read.next_reference_start + 1 if read.next_reference_start >= 0 else 0,
                "TLEN": read.template_length,
                "SEQ": read.query_sequence if read.query_sequence else "*",
                "QUAL": read.qual if read.qual else "*",
            }
            
            # Optional SAM tags (aux fields)
            for tag, value in read.tags:
                record[tag] = value
                all_tags.add(tag)

            records.append(record)
    
    # Build DataFrame
    df = pd.DataFrame(records)

    # Fill missing optional fields with "*"
    df = df.fillna("*")

    # Save CSV
    df.to_csv(output_csv, index=False)
    return output_csv

# Find all BAM files in the folder
bam_files = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.endswith(".bam")]

# Convert all BAM files to CSV
csv_files = []
for bam_file in bam_files:
    csv_file = bam_to_csv(bam_file, output_folder)
    csv_files.append(csv_file)

# Print list of converted CSV files
csv_files

['fastq_7_8_9_10_11_12/5_align_csv/Stepwise_10step_assemble.csv',
 'fastq_7_8_9_10_11_12/5_align_csv/Stepwise_08step_assemble.csv',
 'fastq_7_8_9_10_11_12/5_align_csv/Stepwise_07step_assemble.csv',
 'fastq_7_8_9_10_11_12/5_align_csv/Stepwise_09step_assemble.csv']

## 8.3 Filter Alignments by MAPQ Score

In [77]:
import os
import pandas as pd
from pathlib import Path

# ===== Settings =====
input_dir = Path("fastq_7_8_9_10_11_12/5_align_csv")  # Input folder containing CSV files
output_dir = input_dir / "MAPQ_removed"            # Output folder for filtered CSV files
output_dir.mkdir(parents=True, exist_ok=True)

MAPQ_THRESHOLD = 10     # Keep rows where MAPQ > this value
KEEP_NAN = True         # Keep rows with NaN MAPQ values (e.g., unaligned reads)
# ====================

def process_one_csv(in_path: Path, out_dir: Path, mapq_threshold: int, keep_nan: bool = True):
    out_path = out_dir / in_path.name

    # Remove existing output file to avoid duplicates
    if out_path.exists():
        out_path.unlink()

    # Read input CSV
    try:
        df = pd.read_csv(in_path)
    except Exception as e:
        print(f"⚠️  Read fail: {in_path.name} -> {e}")
        return

    # Skip if MAPQ column does not exist
    if "MAPQ" not in df.columns:
        print(f"⚠️  Skip (no MAPQ column): {in_path.name}")
        return

    # Convert MAPQ column to numeric (invalid entries become NaN)
    m = pd.to_numeric(df["MAPQ"], errors="coerce")

    # Filtering mask: keep MAPQ > threshold, optionally keep NaN
    keep_mask = (m > mapq_threshold) | (m.isna() if keep_nan else False)

    kept = int(keep_mask.sum())
    removed = int((~keep_mask).sum())

    # Save filtered CSV
    df.loc[keep_mask].to_csv(out_path, index=False)
    print(
        f"✅ {in_path.name} → {out_path.name} | kept={kept}, removed={removed} "
        f"| threshold={mapq_threshold}, keep_nan={keep_nan}"
    )

def main():
    csv_files = sorted(input_dir.glob("*.csv"))
    if not csv_files:
        print(f"⚠️  No CSV files in {input_dir}")
        return

    for p in csv_files:
        process_one_csv(p, output_dir, MAPQ_THRESHOLD, KEEP_NAN)

if __name__ == "__main__":
    main()

✅ Stepwise_07step_assemble.csv → Stepwise_07step_assemble.csv | kept=425, removed=12 | threshold=10, keep_nan=True
✅ Stepwise_08step_assemble.csv → Stepwise_08step_assemble.csv | kept=120, removed=7 | threshold=10, keep_nan=True
✅ Stepwise_09step_assemble.csv → Stepwise_09step_assemble.csv | kept=136, removed=25 | threshold=10, keep_nan=True
✅ Stepwise_10step_assemble.csv → Stepwise_10step_assemble.csv | kept=27, removed=17 | threshold=10, keep_nan=True


# Histogram Data Analysis

## A. Generate Histogram Data from Aligned Reads(MAPQ filtered)

In [78]:
import os
import pandas as pd

# Folder setup
input_folder = "fastq_7_8_9_10_11_12/5_align_csv/MAPQ_removed"
histogram_folder = "fastq_7_8_9_10_11_12/6_align_histogram"
os.makedirs(histogram_folder, exist_ok=True)

# Process all CSV files in the input folder
files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

for file_name in files:
    file_path = os.path.join(input_folder, file_name)

    # Clean filename (remove specific substrings)
    clean_name = file_name
    clean_name = clean_name.replace("assemble", "")
    clean_name = clean_name.replace("ID_match_FLASH.extendedFrags", "")
    # remove duplicate/trailing underscores
    clean_name = clean_name.replace("__", "_").strip("_")  
    output_csv = os.path.join(histogram_folder, f"histogram_{clean_name}")

    try:
        df = pd.read_csv(file_path, dtype=str)
        if 'RNAME' not in df.columns:
            print(f"⚠️ Skipping file: {file_name} (no 'RNAME' column found)")
            continue

        # Aggregate and normalize RNAME counts
        rname_counts = df['RNAME'].value_counts().reset_index()
        rname_counts.columns = ['RNAME', 'Count']
        rname_counts.insert(0, 'File_Name', clean_name)
        rname_counts['Count'] = rname_counts['Count'].astype(int)
        total_count = rname_counts['Count'].sum()
        rname_counts['Normalized_Count'] = rname_counts['Count'] / total_count

        rname_counts.to_csv(output_csv, index=False)
        print(f"✅ Saved cleaned RNAME histogram: {output_csv}")

    except Exception as e:
        print(f"❌ Error processing file '{file_name}': {e}")

✅ Saved cleaned RNAME histogram: fastq_7_8_9_10_11_12/6_align_histogram/histogram_Stepwise_09step_.csv
✅ Saved cleaned RNAME histogram: fastq_7_8_9_10_11_12/6_align_histogram/histogram_Stepwise_07step_.csv
✅ Saved cleaned RNAME histogram: fastq_7_8_9_10_11_12/6_align_histogram/histogram_Stepwise_08step_.csv
✅ Saved cleaned RNAME histogram: fastq_7_8_9_10_11_12/6_align_histogram/histogram_Stepwise_10step_.csv


## B. Create Top 5 Histogram Plots for Each Sample

In [79]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# 📁 Folder setup
histogram_folder = "fastq_7_8_9_10_11_12/6_align_histogram"
summary_folder = "fastq_7_8_9_10_11_12/6_align_histogram/graph_top5"
os.makedirs(summary_folder, exist_ok=True)

# 🔴 Highlight mapping (based on filename suffix)
highlight_mapping = {
    "_01step": "seq_0001_1",
    "_02step": "seq_0002_10",
    "_03step": "seq_0005_101",
    "_04step": "seq_0010_1010",
    "_05step": "seq_0021_10101",
    "_06step": "seq_0042_101010",
    "_07step": "seq_0085_1010101",
    "_08step": "seq_0170_10101010",
    "_09step": "seq_0341_101010101",
    "_10step": "seq_0682_1010101010",
    "_11step": "seq_1365_10101010101",
    "_12step": "seq_2730_101010101010",
}

# 📄 List CSV files
csv_files = [f for f in os.listdir(histogram_folder) if f.startswith("histogram_") and f.endswith(".csv")]

# 🔁 Process each file
for file_name in csv_files:
    file_path = os.path.join(histogram_folder, file_name)
    try:
        df = pd.read_csv(file_path)
        if 'RNAME' not in df.columns or 'Normalized_Count' not in df.columns:
            print(f"⚠️ Skipping file: {file_name} (missing column)")
            continue

        # Extract Top 5 RNAMEs
        top_df = df.sort_values(by="Count", ascending=False).head(5).reset_index(drop=True)
        sample_name = file_name.replace("histogram_", "").replace(".csv", "")

        # 🔍 Find highlight RNAME using the suffix
        highlight_rname = None
        for suffix, rname in highlight_mapping.items():
            if suffix in file_name:
                highlight_rname = rname
                break

        # 📊 Create plot
        plt.figure(figsize=(10, 6))
        bars = plt.bar(top_df["RNAME"], top_df["Normalized_Count"], color='blue')

        # 🔴 Color the matched RNAME in red
        for bar, rname in zip(bars, top_df["RNAME"]):
            if rname == highlight_rname:
                bar.set_color('red')

        plt.title(f"Top 5 RNAME Histogram - {sample_name}")
        plt.xlabel("RNAME")
        plt.ylabel("Normalized Count")
        plt.xticks(rotation=45)
        plt.ylim(0, 1)
        plt.tight_layout()

        # 💾 Save
        output_png = os.path.join(summary_folder, file_name.replace(".csv", ".png"))
        output_svg = os.path.join(summary_folder, file_name.replace(".csv", ".svg"))
        plt.savefig(output_png)
        plt.savefig(output_svg)
        plt.close()

        print(f"✅ Saved plot: {output_png}, {output_svg}")

    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")

✅ Saved plot: fastq_7_8_9_10_11_12/6_align_histogram/graph_top5/histogram_Stepwise_07step_.png, fastq_7_8_9_10_11_12/6_align_histogram/graph_top5/histogram_Stepwise_07step_.svg
✅ Saved plot: fastq_7_8_9_10_11_12/6_align_histogram/graph_top5/histogram_Stepwise_10step_.png, fastq_7_8_9_10_11_12/6_align_histogram/graph_top5/histogram_Stepwise_10step_.svg
✅ Saved plot: fastq_7_8_9_10_11_12/6_align_histogram/graph_top5/histogram_Stepwise_09step_.png, fastq_7_8_9_10_11_12/6_align_histogram/graph_top5/histogram_Stepwise_09step_.svg
✅ Saved plot: fastq_7_8_9_10_11_12/6_align_histogram/graph_top5/histogram_Stepwise_08step_.png, fastq_7_8_9_10_11_12/6_align_histogram/graph_top5/histogram_Stepwise_08step_.svg


## C. Summarize Highlighted Read Counts into a CSV File

In [80]:
import os
import pandas as pd
import re

# === Highlight mapping (suffix -> RNAME) ===
highlight_mapping = {
    "_01step": "seq_0001_1",
    "_02step": "seq_0002_10",
    "_03step": "seq_0005_101",
    "_04step": "seq_0010_1010",
    "_05step": "seq_0021_10101",
    "_06step": "seq_0042_101010",
    "_07step": "seq_0085_1010101",
    "_08step": "seq_0170_10101010",
    "_09step": "seq_0341_101010101",
    "_10step": "seq_0682_1010101010",
    "_11step": "seq_1365_10101010101",
    "_12step": "seq_2730_101010101010",
}

# === Folder setup ===
histogram_folder = "fastq_7_8_9_10_11_12/6_align_histogram"
summary_folder = "fastq_7_8_9_10_11_12/7_align_summary"
os.makedirs(summary_folder, exist_ok=True)
highlight_result_csv = os.path.join(summary_folder, "highlight_result.csv")

# === Function to extract step number ===
def extract_step_number(filename):
    match = re.search(r"_(\d+)step", filename)
    return int(match.group(1)) if match else float("inf")

# === Collect highlight summary info ===
highlight_data = []
csv_files = [f for f in os.listdir(histogram_folder) if f.startswith("histogram_") and f.endswith(".csv")]

for file in csv_files:
    file_path = os.path.join(histogram_folder, file)
    try:
        df = pd.read_csv(file_path)
        file_name = file.replace("histogram_", "")

        # Get highlight_rname by suffix
        highlight_rname = ""
        for suffix, rname in highlight_mapping.items():
            if suffix in file_name:
                highlight_rname = rname
                break

        df['Count'] = df['Count'].astype(int)
        total_count = df['Count'].sum()

        highlight_count = df[df['RNAME'] == highlight_rname]['Count'].sum() if highlight_rname else 0
        highlight_percentage = (highlight_count / total_count) * 100 if total_count > 0 else 0

        sorted_counts = df['Count'].sort_values(ascending=False).values
        second_max_count = sorted_counts[1] if len(sorted_counts) >= 2 else (sorted_counts[0] if len(sorted_counts) == 1 else 0)
        highlight_vs_second_ratio = (highlight_count / second_max_count) if second_max_count > 0 else 0

        highlight_data.append([
            file_name,
            highlight_count,
            total_count,
            round(highlight_percentage, 2),
            highlight_rname,
            round(highlight_vs_second_ratio, 3),
            extract_step_number(file_name)
        ])

    except Exception as e:
        print(f"❌ Error processing file '{file}': {e}")

# === Create DataFrame, sort by step, and save ===
highlight_df = pd.DataFrame(highlight_data, columns=[
    'File',
    'Highlight_Count',
    'Total_Count',
    'Highlight_Percentage',
    'Highlight_RNAMEs',
    'Highlight_vs_SecondTop_Ratio',
    'Step_Number'
])

highlight_df = highlight_df.sort_values(by='Step_Number').drop(columns='Step_Number')
highlight_df.to_csv(highlight_result_csv, index=False)

print(f"📌 Highlight summary saved to: {highlight_result_csv}")

📌 Highlight summary saved to: fastq_7_8_9_10_11_12/7_align_summary/highlight_result.csv


## D. Plot Stacked Bar Graph top5_gray_rest_white_box

In [81]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re

# Folder setup
histogram_folder = "fastq_7_8_9_10_11_12/6_align_histogram"
summary_folder = "fastq_7_8_9_10_11_12/7_align_summary"
os.makedirs(summary_folder, exist_ok=True)

# Highlight mapping (based on filename suffix)
highlight_mapping = {
    "_01step": "seq_0001_1",
    "_02step": "seq_0002_10",
    "_03step": "seq_0005_101",
    "_04step": "seq_0010_1010",
    "_05step": "seq_0021_10101",
    "_06step": "seq_0042_101010",
    "_07step": "seq_0085_1010101",
    "_08step": "seq_0170_10101010",
    "_09step": "seq_0341_101010101",
    "_10step": "seq_0682_1010101010",
    "_11step": "seq_1365_10101010101",
    "_12step": "seq_2730_101010101010",
}

# Gray → white gradient color function
def blend_color(base_rgb, t):
    white = np.array([255, 255, 255])
    base = np.array(base_rgb)
    blended = (1 - t) * base + t * white
    return tuple(blended / 255)

base_rgb = (137, 137, 138)

# Extract step number for ascending sort
def extract_step_number(name):
    match = re.search(r'_(\d+)step', name)
    return int(match.group(1)) if match else float('inf')

# Load per-sample data
sample_rname_dfs = {}
for file_name in os.listdir(histogram_folder):
    if file_name.startswith("histogram_") and file_name.endswith(".csv"):
        sample_name = file_name.replace("histogram_", "").replace(".csv", "")
        df = pd.read_csv(os.path.join(histogram_folder, file_name))
        if 'RNAME' not in df.columns or 'Count' not in df.columns:
            continue
        df['Sample'] = sample_name
        df['Count'] = df['Count'].astype(int)
        df['Normalized_Count'] = df['Count'] / df['Count'].sum()
        df = df.sort_values(by='Count', ascending=False).reset_index(drop=True)
        sample_rname_dfs[sample_name] = df

# Sort sample_name by step number
sorted_samples = sorted(sample_rname_dfs.items(), key=lambda x: extract_step_number(x[0]))

# Visualization
fig, ax = plt.subplots(figsize=(24, 12))

for sample_idx, (sample_name, df) in enumerate(sorted_samples):
    # Find highlight RNAME
    highlight_rname = None
    for suffix, rname in highlight_mapping.items():
        if suffix in sample_name:  # check inclusion, not strictly end-of-string
            highlight_rname = rname
            break

    bottom = 0
    top_n = 5
    rest_sum = 0

    for rank, row in df.iterrows():
        rname = row['RNAME']
        height = row['Normalized_Count']

        if rname == highlight_rname:
            ax.bar(sample_name, height, bottom=bottom, color='red', edgecolor='black', linewidth=0.2)
            bottom += height
        elif rank < top_n:
            t = rank / (top_n - 1) if top_n > 1 else 0
            color = blend_color(base_rgb, t)
            ax.bar(sample_name, height, bottom=bottom, color=color, edgecolor='black', linewidth=0.2)
            bottom += height
        else:
            rest_sum += height

    if rest_sum > 0:
        ax.bar(sample_name, rest_sum, bottom=bottom, color='white', edgecolor='black', linewidth=0.2)

# Reference line & styling
ax.axhline(y=0.5, color='gray', linestyle='--', linewidth=1, label='y = 0.5')
ax.set_ylabel("Normalized Count", fontsize=20)
ax.set_xlabel("Sample", fontsize=20)
ax.set_title("Stacked Bar Chart (Red = Highlight, Gray→White = Top 5, Rest = One White Box)", fontsize=16)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Save
png_path = os.path.join(summary_folder, "stacked_bar_top5_gray_rest_white_box.png")
svg_path = os.path.join(summary_folder, "stacked_bar_top5_gray_rest_white_box.svg")
plt.savefig(png_path)
plt.savefig(svg_path)
plt.close()

print(f"✅ Saved:\n - PNG: {png_path}\n - SVG: {svg_path}")

✅ Saved:
 - PNG: fastq_7_8_9_10_11_12/7_align_summary/stacked_bar_top5_gray_rest_white_box.png
 - SVG: fastq_7_8_9_10_11_12/7_align_summary/stacked_bar_top5_gray_rest_white_box.svg
